In [1]:
from deeptables.models import deeptable,deepnets
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

In [2]:
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')

config = deeptable.ModelConfig(nets=deepnets.DCN,auto_discrete=True,metrics=['AUC'])

dt = deeptable.DeepTable(config=config)
oof_proba, eval_proba_mean, test_proba_mean = dt.fit_cross_validation(
    dftrain, 
    y_train, 
    X_test=dfeval, 
    num_folds=5, 
    epochs=50)

Start cross validation
2 class detected, {0, 1}, so inferred as a [binary classification] task
Preparing features taken 0.0024428367614746094s
Imputation taken 0.03056812286376953s
Categorical encoding taken 0.014766931533813477s
Discretization taken 0.0456697940826416s
transform X_test
Iterators:KFold(n_splits=5, random_state=9527, shuffle=True)
Injected a callback [EarlyStopping]. monitor:val_AUC, patience:1, mode:max

Fold:1



/Users/jack/opt/anaconda3/envs/tf_2_0/lib/python3.7/site-packages/sklearn/preprocessing/_discretization.py:202: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
/Users/jack/opt/anaconda3/envs/tf_2_0/lib/python3.7/site-packages/sklearn/preprocessing/_discretization.py:202: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


>>>>>>>>>>>>>>>>>>>>>> Model Desc <<<<<<<<<<<<<<<<<<<<<<< 
---------------------------------------------------------
inputs:
---------------------------------------------------------
['all_categorical_vars: (9)', 'input_continuous_all: (4)']
---------------------------------------------------------
embeddings:
---------------------------------------------------------
input_dims: [4, 5, 10, 6, 4, 5, 4, 4, 6]
output_dims: [4, 4, 4, 4, 4, 4, 4, 4, 4]
dropout: 0.3
---------------------------------------------------------
dense: dropout: 0
batch_normalization: False
---------------------------------------------------------
concat_embed_dense: shape: (None, 40)
---------------------------------------------------------
nets: ['dcn_nets']
---------------------------------------------------------
dcn-widecross: input_shape (None, 40), output_shape (None, 40)
dcn-dnn2: input_shape (None, 40), output_shape (None, 64)
dcn: input_shape (None, 40), output_shape (None, 104)
--------------------------

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  2.0min finished


In [3]:
dt.leaderboard

,model,type,loss,*auc,val_loss,val_auc
0,dcn_nets-kfold-1,val,0.445111,0.857538,0.793008,0.711317
1,dcn_nets-kfold-4,val,0.452028,0.851079,0.618186,0.771930
2,dcn_nets-kfold-5,val,0.458760,0.849411,0.650901,0.781089
3,dcn_nets-kfold-2,val,0.483778,0.828541,0.583975,0.755244
4,dcn_nets-kfold-3,val,0.550232,0.782319,0.723225,0.620265


In [4]:
# oof score
roc_auc_score(y_train, oof_proba)

0.7150098593964335

In [5]:
# test score
roc_auc_score(y_eval,test_proba_mean)

0.7259565350474442